<a href="https://colab.research.google.com/github/lyubovchubarova/NLP_HW/blob/main/HW2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
from collections import defaultdict as dd
from pymorphy2 import MorphAnalyzer
import stanza
import spacy
import natasha

**Разметка подготовленного мини-корпуса.** В качестве корпусного материала выбран отрывок из поэмы "Победа над Солнцем" Алексея Крученых. В этой поэме он активно занимается словотворчеством и кроме того, в некоторых отрывках смешивает неологизмы по частям речи. Так, у него только по контексту можно понять разницу между существительным и кратким прилагательным. Ниже привожу отрывок из текста в качестве примера

*Созерцавель поведет вас*,

*Созерцебен есть вождебен*,

*Сборище мрачных вождей*

*От мучав и ужасавлей до веселян и нездешних смеяв и веселогов пройдут перед внимательными видухами и созерцалями и глядарями: мина вы, бывавы, певавы, бытавы, идуньи, зовавы, величавы, судьбоспоры и малюты.* 

*Зовавы позовут вас, как и полунебесные оттудни.*

*Минавы расскажут вам, кем вы были некогда.*

*Бытавы – кто вы, бывавы – кем вы могли быть.*

*Малюты утроги и утравы расскажут, кем будете.*

*Никогдавли пройдут, как тихое сновидение.*


In [ ]:
punct = string.punctuation
tagged_corpora = {}

with open('corpora.txt', encoding = 'utf-8') as f:
  lines = f.readlines()
  for line in lines:
      for elem in punct:
          if elem in line:
              line = line.replace(elem, '')
      line = line.split()
      for elem in line:
          elem = elem.lower()
          print(elem)
           tagged_corpora[elem] = input()

созерцавель
NOUN
поведет
VERB
вас
PRON
созерцебен
NOUN
есть
AUX
вождебен
ADJ
сборище
NOUN
мрачных
ADJ
вождей
NOUN
от
ADP
мучав
NOUN
и
CONJ
ужасавлей
NOUN
до
ADP
веселян
NOUN
и
CONJ
нездешних
ADJ
смеяв
NOUN
и
CONJ
веселогов
NOUN
пройдут
VERB
перед
ADP
внимательными
ADJ
видухами
NOUN
и
CONJ
созерцалями
NOUN
и
CONJ
глядарями
NOUN
мина
NOUN
вы
PRON
бывавы
ADJ
певавы
ADJ
бытавы
ADJ
идуньи
ADJ
зовавы
ADJ
величавы
ADJ
судьбоспоры
ADJ
и
CONJ
малюты
NOUN
зовавы
NOUN
позовут
VERB
вас
PRON
как
SCONJ
и
INTJ
полунебесные
ADJ
оттудни
NOUN
минавы
NOUN
расскажут
VERB
вам
PRON
кем
SCONJ
вы
PRON
были
VERB
некогда
ADV
бытавы
NOUN
–
SYM
кто
SCONJ
вы
PRON
бывавы
NOUN
–
SYM
кем
SCONJ
вы
PRON
могли
VERB
быть
VERB
малюты
NOUN
утроги
ADJ
и
CONJ
утравы
ADJ
расскажут
VERB
кем
SCONJ
будете
VERB
никогдавли
NOUN
пройдут
VERB
как
SCONJ
тихое
ADJ
сновидение
NOUN
маленькие
ADJ
повелюты
NOUN
властно
ADV
поведут
VERB
вас
PRON
здесь
ADV
будут
AUX
иногдавли
NOUN
и
CONJ
воображавли
NOUN
а
CONJ
с
ADP
ними
PRON
сно
NOUN
и
CO

In [89]:
 corpora = list(tagged_corpora.keys())
 gold = list(tagged_corpora.values())

In [105]:
#POS-tagging  с помощью spacy
spacy_tags = []
corpora_str = ' '.join(corpora)

nlp = spacy.load('ru_core_news_sm')
doc = nlp(corpora_str)

for token in doc:
  spacy_tags.append(token.pos_)


In [106]:
#POS-tagging с помощью Natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [116]:
natasha_tags = []
text = corpora_str
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
  if token.text == 'ые':
    continue
  else:
    natasha_tags.append(token.pos)
#дело в том, что Наташа размечает токен '1ые', как два токена - '1' и 'ые', а остальные тэггеры этого не делают

In [99]:
#POS-tagging с помощью pymorphy
#Две библиотеки выше размечают по стандарту UPOS, у pymorphy свой список обозначений, поэтому создадим словарь соответствий


pymorphy_tags = []
pymorphy_upos = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADV', 'INFN': 'VERB', 'PRTF':'VERB', 'PRTS': 'VERB', 'GRND':'VERB', 'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO': 'PRON', 'PRED':'VERB', 'PREP': 'ADP', 'PRCL': 'PART', None: 'X'}

m = MorphAnalyzer()

for token in corpora:
  pos = m.parse(token)[0].tag.POS
  if pos in pymorphy_upos:
    pymorphy_tags.append(pymorphy_upos[pos])
  else:
    pymorphy_tags.append(pos)

In [117]:
#Сравнение точночти разметки.
from sklearn.metrics import accuracy_score

print('Точность разметки spacy - ' + str(accuracy_score(gold, spacy_tags)))
print('Точность разметки natasha - ' + str(accuracy_score(gold, natasha_tags)))
print('Точность разметки pymorphy - ' + str(accuracy_score(gold, pymorphy_tags)))

Точность разметки spacy - 0.7674418604651163
Точность разметки natasha - 0.6337209302325582
Точность разметки pymorphy - 0.7674418604651163


С помощью spacy пишу chunker для следующих сочетаний:
1. не + VERB
2. ADJ + NOUN
3. VERB + ADP + NOUN

In [186]:
def chunker(data):
  ne_verb = []
  adj_noun = []
  verb_adp_noun = []

  nlp = spacy.load('ru_core_news_sm')
  doc = nlp(data)
  for token in doc:
    #не + VERB
    if token.text.lower() == 'не' and token.head.pos_ == 'VERB':
      ne_verb.append(token.text + ' ' + token.head.text)

    #ADJ + NOUN
    if token.pos_ == 'ADJ' and token.head.pos_ == 'NOUN':
      adj_noun.append(token.text + ' '  + token.head.text )

    # VERB + ADP + NOUN
    if (token.pos_ == 'NOUN' or token.pos_ == 'PROP' or token.pos_ == 'PRON') and token.head.pos_ == 'VERB':
      for child in token.children:
        if child.pos_ == 'ADP':
         verb_adp_noun.append(token.head.text + ' ' + child.text + ' ' + token.text)

  return ne_verb, adj_noun, verb_adp_noun

In [187]:
#Можно затестить функцию на рассказе Паустовского "Телеграмма"

with open('Телеграмма.txt', encoding='utf-8') as f:
  text = f.read()

  ne_verb, adj_noun, verb_adp_noun = chunker(text)

In [190]:
print(ne_verb[0:3], adj_noun[0:3], verb_adp_noun[0:3])

['не мог', 'не мог', 'не знала'] ['холодный редкость', 'Тесовые крыши', 'Спутанная трава'] ['осыпаться у забора', 'тащились Над лугами', 'тащились из реки']


Для улучшения качества определения тональности, я бы взяла шаблон не + любое слово с положительной окраской из сета таковых. Очень часто в негативных отзывах встречались сочетания типа "не красивый", "дошел не быстро" и проч. Аналогично можно сделать с не + словами с негативной окраской  - ср. "не дорогой"